### [803 打砖块](https://leetcode.cn/problems/bricks-falling-when-hit/description/)
<font color=#FF3300>困难</font>  
连通性判定 <font color=#00CCFF>并查集</font>

首先定义并查集

In [1]:
import copy


class UnionFind:
    def __init__(self, size: int):
        # 当前结点的父亲结点
        self._parent = [i for i in range(size)]
        # 以当前结点为根结点的子树的结点总数
        self._son_size = [1] * size

    def find(self, x: int) -> int:
        parent = self._parent
        if parent[x] != x:
            parent[x] = self.find(parent[x])
        return parent[x]

    def merge(self, x: int, y: int):
        x_parent, y_parent = self.find(x), self.find(y)
        if x_parent == y_parent:
            return
        self._parent[x_parent] = y_parent
        # 归并子树的数量
        self._son_size[y_parent] += self._son_size[x_parent]

    def son_size(self, x: int) -> int:
        parent = self.find(x)
        return self._son_size[parent]


In [5]:
from typing import List


class Solution:
    def hitBricks(self, grid: List[List[int]], hits: List[List[int]]) -> List[int]:
        row = len(grid)
        col = len(grid[0])

        copy_grid = copy.deepcopy(grid)

        # 0 - 空白(无砖块) 1 - 有砖块
        # 把 copy过来的格子先全部击碎
        for i, j in hits:
            copy_grid[i][j] = 0

        # 将二维数组映射到一维数组中，坐标变化为: (x,y)-> x * col + y
        # 对于row, col的二维数组，则需要总的一维数组的长度为 row * col.
        # 外加一个特殊节点X，将其一维下标定为 row * col + 1
        union_find = UnionFind(row * col + 1)

        top_idx = row * col
        # 将顶行(也就是y坐标为0的节点) 和 特殊节点 X(row * col + 1) 相连
        for j in range(col):
            if copy_grid[0][j] == 1:
                union_find.merge(j, top_idx)

        # 其余网格，如果是砖块，做一下前后左右的合并
        for i in range(1, row):
            for j in range(0, col):
                if copy_grid[i][j] == 1:
                    if copy_grid[i - 1][j] == 1:
                        union_find.merge((i - 1) * col + j, i * col + j)
                    if j > 0 and copy_grid[i][j - 1] == 1:
                        union_find.merge(i * col + j - 1, i * col + j)

        res = [0] * len(hits)

        directions = (1, 0), (-1, 0), (0, 1), (0, -1)

        for i in range(len(hits) - 1, -1, -1):
            x, y = hits[i]

            if grid[x][y] == 0:
                continue

            original_size = union_find.son_size(top_idx)

            if x == 0:
                union_find.merge(x * col + y, top_idx)

            for dx, dy in directions:
                if 0 <= x + dx < row and 0 <= y + dy < col and copy_grid[x + dx][y + dy] == 1:
                    union_find.merge(x * col + y, (x + dx) * col + y + dy)

            current_size = union_find.son_size(top_idx)

            res[i] = max(0, current_size - original_size - 1)
            copy_grid[x][y] = 1
        return res


if __name__ == '__main__':
    print(Solution().hitBricks(grid=[[1, 0, 0, 0], [1, 1, 1, 0]], hits=[[1, 0]]))

[0]
